# Apriori

In [1]:
!pip install mlxtend --upgrade

     |████████████████████████████████| 1.3 MB 2.7 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [2]:
import mlxtend 
mlxtend.__version__

'0.19.0'

In [3]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
import pandas as pd
import os

## Dataset import 및 확인

In [5]:
from google.colab import drive
drive.mount('/content/drive')

basicpath = '/content/drive/MyDrive/'

Mounted at /content/drive


In [9]:
path = basicpath + 'Univ/4-1/Data Science/Practice/2주차'
file = 'BreadBasket_DMS.csv'
data = pd.read_csv(os.path.join(path, file), index_col = None)

ParserError: ignored

In [7]:
data.head(10)

,7\t14
0,9
1,18\t2\t4\t5\t1
2,1\t11\t15\t2\t7\t16\t4\t13
3,2\t1\t16
4,15\t7\t6\t11\t18\t9\t12\t19\t14
5,11\t2\t13\t4
6,11\t13
7,7\t4\t2\t17\t19\t3\t8\t16\t1
8,18\t16\t15\t10\t2\t8\t6\t0\t4\t5
9,9


## Data cleaning

### 편의를 위해 소문자로 변환

In [ ]:
data['Item'] = data['Item'].str.lower()
data.head(3)

,Date,Time,Transaction,Item
0,2016.10.30,9:58:11,1,bread
1,2016.10.30,10:05:34,2,scandinavian
2,2016.10.30,10:05:34,2,scandinavian


### Data의 none value 제거

In [ ]:
data.isnull().sum() # 결측치 제거 나중에 볼거임 지금 신경 X

Date           0
Time           0
Transaction    0
Item           0
dtype: int64

In [ ]:
data.iloc[25:30]

,Date,Time,Transaction,Item
25,2016.10.30,10:27:21,11,bread
26,2016.10.30,10:27:21,11,none
27,2016.10.30,10:30:14,12,jam
28,2016.10.30,10:30:14,12,coffee
29,2016.10.30,10:30:14,12,tartine


In [ ]:
data.iloc[26]

Date           2016.10.30
Time             10:27:21
Transaction            11
Item                 none
Name: 26, dtype: object

In [ ]:
(data['Item'] == 'none').value_counts()

False    20507
True       786
Name: Item, dtype: int64

In [ ]:
len(data)

21293

In [ ]:
data = data.drop(data[data['Item'] == 'none'].index)

In [ ]:
len(data)

20507

In [ ]:
len(data['Item'].unique())

94

## Data 간단 분석_item당 등장횟수

In [ ]:
top10_items = data['Item'].value_counts().head(20)
top10_items

coffee           5471
bread            3325
tea              1435
cake             1025
pastry            856
sandwich          771
medialuna         616
hot chocolate     590
cookies           540
brownie           379
farm house        374
muffin            370
alfajores         369
juice             369
soup              342
scone             327
toast             318
scandinavian      277
truffles          193
coke              185
Name: Item, dtype: int64

## Apriori 활용

### Data형태 변환: Vertical format

In [ ]:
hot_encoded_data = data.groupby(['Transaction','Item'])['Item'].count().unstack().fillna(0)
# 동일한 Transaction 내에서 각각의 Item 개수를 센거야
# database의 groupby랑 똑같아
# 다음시간엔 unstack 가르쳐준대

In [ ]:
hot_encoded_data.head(9)

Item,adjustment,afternoon with the baker,alfajores,argentina night,art tray,bacon,baguette,bakewell,bare popcorn,basket,...,the bart,the nomad,tiffin,toast,truffles,tshirt,valentine's card,vegan feast,vegan mincepie,victorian sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#하나의 transaction에 2개 이상의 동일한 item이 들어간 경우 1개로 취급. 
#item 간의 관계가 중요. 동일한 item끼리는 중요하지 않음
def encode_units(x): 
    if x <= 0:
        return 0
    if x >= 1:
        return 1

hot_encoded_data = hot_encoded_data.applymap(encode_units)

### Apriori

In [ ]:
frequent_itemsets = apriori(hot_encoded_data, min_support=0.01, use_colnames=True)

In [ ]:
frequent_itemsets

,support,itemsets
0,0.036344,(alfajores)
1,0.016059,(baguette)
2,0.327205,(bread)
3,0.040042,(brownie)
4,0.103856,(cake)
...,...,...
56,0.023666,"(toast, coffee)"
57,0.014369,"(tea, sandwich)"
58,0.010037,"(bread, cake, coffee)"
59,0.011199,"(bread, coffee, pastry)"


In [ ]:
# rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.4)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.15)


In [ ]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(alfajores),(bread),0.036344,0.327205,0.010354,0.284884,0.870657,-0.001538,0.940818
1,(alfajores),(coffee),0.036344,0.478394,0.019651,0.540698,1.130235,0.002264,1.135648
2,(brownie),(bread),0.040042,0.327205,0.010777,0.269129,0.822508,-0.002326,0.920538
3,(cake),(bread),0.103856,0.327205,0.023349,0.224822,0.687097,-0.010633,0.867923
4,(bread),(coffee),0.327205,0.478394,0.090016,0.275105,0.575059,-0.066517,0.719561
5,(coffee),(bread),0.478394,0.327205,0.090016,0.188163,0.575059,-0.066517,0.828731
6,(cookies),(bread),0.054411,0.327205,0.014474,0.266019,0.813004,-0.003329,0.916638
7,(hot chocolate),(bread),0.058320,0.327205,0.013418,0.230072,0.703144,-0.005665,0.873841
8,(medialuna),(bread),0.061807,0.327205,0.016904,0.273504,0.835879,-0.003319,0.926082
9,(pastry),(bread),0.086107,0.327205,0.029160,0.338650,1.034977,0.000985,1.017305


In [ ]:
rules[rules['confidence'] > 0.17]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(alfajores),(bread),0.036093,0.324940,0.010282,0.284884,0.876728,-0.001446,0.943987
1,(alfajores),(coffee),0.036093,0.475081,0.019515,0.540698,1.138116,0.002368,1.142861
2,(brownie),(bread),0.039765,0.324940,0.010702,0.269129,0.828244,-0.002219,0.923639
3,(cake),(bread),0.103137,0.324940,0.023187,0.224822,0.691888,-0.010326,0.870846
4,(bread),(coffee),0.324940,0.475081,0.089393,0.275105,0.579069,-0.064980,0.724131
5,(coffee),(bread),0.475081,0.324940,0.089393,0.188163,0.579069,-0.064980,0.831522
6,(cookies),(bread),0.054034,0.324940,0.014374,0.266019,0.818673,-0.003184,0.919725
7,(hot chocolate),(bread),0.057916,0.324940,0.013325,0.230072,0.708047,-0.005494,0.876784
8,(medialuna),(bread),0.061379,0.324940,0.016787,0.273504,0.841708,-0.003157,0.929201
9,(none),(bread),0.079005,0.324940,0.020564,0.260292,0.801048,-0.005107,0.912604


## Apriori 알고리즘과의 성능 비교

In [ ]:
%timeit -n 100 -r 10 frequent_itemsets_apriori = apriori(hot_encoded_data, min_support=0.005, use_colnames=True)

100 loops, best of 10: 138 ms per loop


In [ ]:
%timeit -n 100 -r 10 frequent_itemsets = fpgrowth(hot_encoded_data, min_support=0.005, use_colnames=True)

100 loops, best of 10: 83.1 ms per loop
